# Examples with a Postgres DB

> **These are Go notebooks**: In order to use the GoNB Jupyter Kernel, please install GoNB from here: [https://github.com/janpfeifer/gonb](https://github.com/janpfeifer/gonb)

Note also that for local package development, you can put: `!*go mod edit -replace "github.com/umbralcalc/stochadex=/path/to/stochadex"` at the top of any cell.

## Postgres writing

(see `Dockerfile.postgres` for an example DB setup)

In [6]:
import (
	"github.com/umbralcalc/stochadex/pkg/simulator"
	"github.com/umbralcalc/stochadex/pkg/analysis"
	"github.com/umbralcalc/stochadex/pkg/continuous"
	"github.com/umbralcalc/stochadex/pkg/discrete"
)

%%

// Create a simulator.StateTimeStorage from a simulation run
storage := analysis.NewStateTimeStorageFromPartitions(
	// Instantiate the desired simulation state partitions
	[]*simulator.PartitionConfig{
		{
			Name:              "gbm",
			Iteration:         &continuous.GeometricBrownianMotionIteration{},
			Params:            simulator.NewParams(map[string][]float64{
				"variances": {2.0, 3.0, 1.0},
			}),
			InitStateValues:   []float64{1.0, 1.0, 1.0},
			StateHistoryDepth: 1,
			Seed:              5678,
		},
		{
			Name:              "group_numbers",
			Iteration:         &discrete.BinomialObservationProcessIteration{},
			Params:            simulator.NewParams(map[string][]float64{
				"observed_values": {5, 5, 5},
				"state_value_observation_probs": {0.1, 0.4, 0.7},
				"state_value_observation_indices": {0, 1, 2},
			}),
			InitStateValues:   []float64{0.0, 0.0, 0.0},
			StateHistoryDepth: 1,
			Seed:              321,
		},
    },
	// Decide when should we stop the simulation
	&simulator.NumberOfStepsTerminationCondition{
		MaxNumberOfSteps: 1000,
	},
	// Decide how time should evolve
	&simulator.ConstantTimestepFunction{
		Stepsize: 1.0,
	},
	// Input the initial time
	0.0,
)

// Specify the Postgres DB configuration
db := &analysis.PostgresDb{
	User: "stochadexuser",
	Password: "stochadexpassword",
	Dbname: "stochadexdb",
	TableName: "testnumbers",
}

// Write the data to the configured Postgres DB
analysis.WriteStateTimeStorageToPostgresDb(db, storage)

## Postgres querying

Create scatterplots

In [ ]:
import (
	"github.com/umbralcalc/stochadex/pkg/analysis"
)

%%

// Specify the Postgres DB configuration
db := &analysis.PostgresDb{
	User: "stochadexuser",
	Password: "stochadexpassword",
	Dbname: "stochadexdb",
	TableName: "testnumbers",
}

storage, _ := analysis.NewStateTimeStorageFromPostgresDb(
	db,
	[]string{"gbm", "group_numbers"},
	0.0,
	1000.0,
)

fmt.Println(storage.GetValues("gbm"))

## Grouped aggregations

In [5]:
import (
	"github.com/umbralcalc/stochadex/pkg/analysis"

	"github.com/go-echarts/go-echarts/v2/opts"
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

%%

// Specify the Postgres DB configuration
db := &analysis.PostgresDb{
	User: "stochadexuser",
	Password: "stochadexpassword",
	Dbname: "stochadexdb",
	TableName: "testnumbers",
}

storage, _ := analysis.NewStateTimeStorageFromPostgresDb(
	db,
	[]string{"gbm", "groups"},
	0.0,
	1000.0,
)

groupedStorage := analysis.NewGroupedStateTimeStorage(
	analysis.AppliedGrouping{
		GroupBy: []analysis.DataRef{{PartitionName: "groups"}},
		Default: 0,
		Precision: 1,
	},
	storage,
)

groupedSumPartition := analysis.NewGroupedAggregationPartition(
	general.SumAggregation,
	analysis.AppliedAggregation{
		Name: "grouped_sum",
		Data: analysis.DataRef{PartitionName: "gbm"},
	},
	groupedStorage,
)

// Run and add the grouped sum partition to storage
storage = analysis.AddPartitionToStateTimeStorage(
	storage,
	groupedSumPartition,
	map[string]int{"gbm": 1, "groups": 1},
)

// Reference the plotting data for the x-axis
xRef := analysis.DataRef{IsTime: true}

// Reference the plotting data for the y-axis
yRefs := []analysis.DataRef{{PartitionName: "grouped_sum"}}

// Create a scatter plot from partitions in a simulator.StateTimeStorage
scatter := analysis.NewScatterPlotFromPartition(storage, xRef, yRefs)

// Display the plot in a Go notebook
gonb_echarts.Display(scatter, "width: 1024px; height:400px; background: white;")


failed to run "/home/robert/Code/go/pkg/mod/golang.org/toolchain@v0.0.1-go1.23.2.linux-amd64/bin/go get": exit status 1 

 


 

 


 go: module github.com/janpfeifer/gonb-echarts: Get "https://proxy.golang.org/github.com/janpfeifer/gonb-echarts/@v/list": dial tcp: lookup proxy.golang.org on 127.0.0.53:53: server misbehaving

ERROR: failed to run "/home/robert/Code/go/pkg/mod/golang.org/toolchain@v0.0.1-go1.23.2.linux-amd64/bin/go get": exit status 1